In [1]:
import boto3
from project3_iam_user import *

In [2]:
DWH_CLUSTER_TYPE = 'single-node'
DWH_NUM_NODES = 2
DWH_NODE_TYPE = 'dc2.large'
DWH_CLUSTER_IDENTIFIER = 'project3-cluster'
DWH_DB = 'project3_db'
DWH_DB_USER = 'p3user'
DWH_DB_PASSWORD = 'p3Passw0rd'
DWH_PORT = 5439
DWH_IAM_ROLE_NAME =  'project3_redshift_role'


# Create clients for IAM and Redshift

In [3]:
iam = boto3.client('iam',aws_access_key_id=ACCESS_KEY_ID,
                     aws_secret_access_key=SECRET_ACCESS_KEY,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=ACCESS_KEY_ID,
                       aws_secret_access_key=SECRET_ACCESS_KEY
                       )

# Create IAM Role for Redshift
Make Redshift able to read s3

In [4]:
from botocore.exceptions import ClientError
import json

#Create the role, 
try: #it might exists already
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
dwh_role_arn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(dwh_role_arn)

1.1 Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name project3_redshift_role already exists.
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::929805762734:role/project3_redshift_role


# Create Redshift cluster
Create a Redshift cluster using the preset parameters and associate the created role to it

In [5]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType = DWH_CLUSTER_TYPE,
        NodeType = DWH_NODE_TYPE,
        #NumberOfNodes = DWH_NUM_NODES,

        #Identifiers & Credentials
        DBName = DWH_DB,
        ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
        MasterUsername = DWH_DB_USER,
        MasterUserPassword = DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[dwh_role_arn]  
    )
except Exception as e:
    print(e)



# Get cluster properties

In [6]:
# wait until the cluster is ready
waiter = redshift.get_waiter('cluster_available')
waiter.wait( 
    ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
)

#get properties dictionary
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

# Write cluster props to cfg file

In [7]:
file_text = f"""[CLUSTER]
HOST='{myClusterProps['Endpoint']['Address']}'
DB_NAME='{DWH_DB}'
DB_USER='{DWH_DB_USER}'
DB_PASSWORD='{DWH_DB_PASSWORD}'
DB_PORT='{myClusterProps['Endpoint']['Port']}'

[IAM_ROLE]
ARN='{dwh_role_arn}'

[S3]
LOG_DATA='s3://udacity-dend/log_data'
LOG_JSONPATH='s3://udacity-dend/log_json_path.json'
SONG_DATA='s3://udacity-dend/song_data'
"""

f = open('dwh.cfg', 'w')
f.write(file_text)
f.close()

# Delete cluster

In [8]:
redshift.delete_cluster( ClusterIdentifier='project3-cluster',  SkipFinalClusterSnapshot=True)

{'Cluster': {'ClusterIdentifier': 'project3-cluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'p3user',
  'DBName': 'project3_db',
  'Endpoint': {'Address': 'project3-cluster.cqh4kd6a34p7.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2021, 12, 6, 0, 19, 27, 827000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0006eb2f148068d87',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-0ab9503a980b66838',
  'AvailabilityZone': 'us-west-2c',
  'PreferredMaintenanceWindow': 'sun:11:00-sun:11:30',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'Numb